## Example: Various initilizations of the Executor

This example shows different ways to initialize the Executor und run various jobs.

In [1]:
from qiskit import Aer
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.random import random_circuit

from squlearn.util import Executor

#import logging
#logging.basicConfig(level=logging.INFO, filename="qiskit.log", filemode="w")

Different initializations of the ``Executor`` class from different backends or services.

In [2]:
# from a string representing the simple Qiskit simulators:
executor = Executor("statevector_simulator")
executor = Executor("qasm_simulator")

# from a given backend:
executor = Executor(Aer.get_backend("statevector_simulator"))

# # from a Service and the backend (here, a token has been placed in qiskitrc):
# from qiskit_ibm_runtime import QiskitRuntimeService
# service = QiskitRuntimeService(channel="ibm_quantum")
# # Alternative: service = QiskitRuntimeService(channel="ibm_quantum",token="YOUR_TOKEN_HERE")
# executor = Executor(service.get_backend('ibm_nairobi'))

# # from a session
# from qiskit_ibm_runtime import Session
# session = Session(service, backend=service.get_backend('ibm_nairobi'),max_time=28800)
# executor = Executor(session)

# # from the qiskit-ibm-runtime Estimator
# from qiskit_ibm_runtime import Estimator
# estimator = Estimator(session)
# executor = Executor(estimator)

# from a Qiskit primitive:
from qiskit.primitives import Sampler, Estimator, BackendSampler, BackendEstimator
executor = Executor(Estimator())
executor = Executor(Sampler())
executor = Executor(BackendEstimator(Aer.get_backend("qasm_simulator")))
executor = Executor(BackendSampler(Aer.get_backend("qasm_simulator"), {"shots": 1000}))

In [3]:
# Shots can be set by the executor:
print("Current shots as set before:", executor.get_shots())
# Set shots
executor.set_shots(1234)
print("Adjusted shots:", executor.get_shots())
# Reset shots to initial ones:
executor.reset_shots()
print("Reset shots:", executor.get_shots())

Current shots as set before: 1000
Adjusted shots: 1234
Reset shots: 1000


In [4]:
# The executor generates modified Primitives that can be used as usual:
# The modified primitives allow caching, automatic session handling and logging.

# Generate a random circuit:
circuit = random_circuit(2, 2, seed=0).decompose(reps=1)

# Generate a observable:
observable = SparsePauliOp("ZI")

# Get the Executor Estimator Primitive and call run:
estimator = executor.get_estimator()
print(estimator.run(circuit, observable, shots=4321).result())

# Get the Executor Sampler Primitive and call run:
sampler = executor.get_sampler()
print(sampler.run(circuit.measure_all(inplace=False)).result())

circuit.measure_all()

EstimatorResult(values=array([-0.13029391]), metadata=[{'variance': 0.9830234961189362, 'shots': 4321}])
SamplerResult(quasi_dists=[{3: 0.509, 2: 0.044, 0: 0.411, 1: 0.036}], metadata=[{'shots': 1000}])


In [5]:
# The executor can also be used to execute backend.run() (caching not implemented yet)
job = executor.backend_run(circuit)
job.result()

Result(backend_name='qasm_simulator', backend_version='0.12.0', qobj_id='', job_id='3782e27c-0c8a-47f6-8377-9ce8c0927b0b', success=True, results=[ExperimentResult(shots=1000, success=True, meas_level=2, data=ExperimentResultData(counts={'0x3': 516, '0x0': 397, '0x1': 40, '0x2': 47}), header=QobjExperimentHeader(creg_sizes=[['meas', 2]], global_phase=0.0, memory_slots=2, metadata=None, n_qubits=2, name='circuit-131', qreg_sizes=[['q', 2]]), status=DONE, seed_simulator=4094219017, metadata={'batched_shots_optimization': False, 'method': 'statevector', 'active_input_qubits': [0, 1], 'device': 'CPU', 'remapped_qubits': False, 'num_qubits': 2, 'num_clbits': 2, 'sample_measure_time': 0.0002881, 'input_qubit_map': [[0, 0], [1, 1]], 'measure_sampling': True, 'noise': 'ideal', 'parallel_shots': 1, 'parallel_state_update': 12, 'fusion': {'enabled': True, 'threshold': 14, 'max_fused_qubits': 5, 'applied': False}}, time_taken=0.0015462)], date=2023-09-28T13:29:35.499427, status=COMPLETED, header=N

In [6]:
# The executor features a log of what is done in the background (useful for real backends):

executor = Executor("qasm_simulator", log_file="example_log.log")
executor.set_shots(1234)
estimator = executor.get_estimator()

#j=Estimator().run(circuit, observable, shots=4321)
#j.result()

print(estimator.run(circuit, observable, shots=4321).result())

QiskitError: 'Cannot apply instruction with classical bits: measure'

In [ ]:
# The executor features a cache where the result of jobs are stored and can be reused:

executor = Executor(
    BackendSampler(Aer.get_backend("qasm_simulator")),
    log_file="example_log_cache.log",
    caching=True,
    cache_dir="_cache",
)
executor.set_shots(1234)
estimator = executor.get_estimator()
print(estimator.run(circuit, observable, shots=4321).result())
print(estimator.run(circuit, observable, shots=1234).result())
print(estimator.run(circuit, observable, shots=4321).result())